In [4]:
import langchain
from langchain_community.document_loaders import DataFrameLoader
import json
import pandas as pd
import getpass
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings 
import openai 
from langchain_core.documents import Document

In [5]:
with open('../data/subject_areas.json', 'r') as f:
    subject_areas = json.load(f)

In [6]:
set_of_subjects = set()
set_of_new_subjects = set()
for subject in subject_areas:
    set_of_subjects.add(subject['code'])
    subject['code'] = subject['code'].replace(' ', '')
    set_of_new_subjects.add(subject['code'])



print(len(set_of_subjects))
print(len(set_of_new_subjects))

469
469


In [14]:
print(subject_areas[0]['code'])

AA


In [7]:
subject_dict = [{k: v for k, v in d.items() if k in ['code', 'title'] } for d in subject_areas]

In [8]:
print(subject_dict[1])
print(len(subject_dict))

{'code': 'AE', 'title': 'AEROSPACE ENGINEERING'}
469


In [9]:
weed_out = ['BOTHELL', 'TACOMA', 'MASTERS']

In [10]:
abbrv_desc = [f'{title}: {code}' for code, title in zip([d['code'] for d in subject_dict], [d['title'] for d in subject_dict]) if not any(weed in title for weed in weed_out)]

In [11]:
print(len(abbrv_desc))

367


In [12]:
#Setting up keys
pinecone_api_key = os.environ.get("pinecone_API")

pc = Pinecone(api_key=pinecone_api_key)

api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key


In [13]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

pc.create_index(
    name= 'department-abbrev-db',
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)



PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '0a438270289567713e113240ddb95a8e', 'Date': 'Mon, 17 Mar 2025 06:34:48 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [14]:
index = pc.Index("department-abbrev-db")
vector_store = PineconeVectorStore(index , embedding = embeddings)

In [15]:
vector_store.delete(delete_all = True)

In [6]:
index = pc.Index("department-abbrev-db")

In [16]:
documents = [Document(page_content = doc) for doc in abbrv_desc]
print(documents[0])

page_content='AERONAUTICS & ASTRONAUTICS: AA'


In [17]:

vector_store.add_documents(documents)



['c120767e-4cf7-499d-826b-85650b1018b0',
 'a0d62e64-e36e-4409-81ec-2aae4f226392',
 '2d5c80da-a6fb-4095-9ca4-b64df3e07f51',
 '594fb598-d337-4f34-85cf-cfca443de0ad',
 'a69712fb-3179-4d54-ac20-1cb094e353e4',
 'cdbddfde-df53-4250-a5a4-1f00699ca819',
 '024f8e0b-f91e-4195-9a7c-787ad55238d6',
 '69d31858-b543-402c-8aac-80cb00249582',
 'd76f8919-b5da-406d-9a8f-065149a9f983',
 'e88b9c45-e5f0-40b8-a824-7ed0ad593951',
 '04554ee8-e78c-4948-908c-b0eb34238816',
 '811f5413-2fd8-43d3-9a0f-646d329e6137',
 '27834c21-f208-4200-a563-3e0d521989a5',
 'ef2a063f-e293-4969-9abc-a50f0af272e3',
 '2edb61ea-a29a-4ec0-9629-0a72badea2b6',
 '2ac0b0d3-b1c2-4d2d-8bfd-274244be0568',
 '8bfcec6e-ec19-4e60-b174-04ee71fbc519',
 '3c772ca9-d509-4f88-9224-743400c5c718',
 '9ea414d2-5146-482e-ad40-8daee43fdfcd',
 '590e6eb6-2ce8-4f88-bbfd-9b63939217ce',
 'fd07e5a7-3812-46bd-976b-0dd874625759',
 'df741995-bfd3-4088-bf81-6b0f186b7208',
 '8c891b81-0eec-42f2-9bae-7a136409a92d',
 '887297fa-fcd5-441f-8a6a-1d787cbd856e',
 '16330203-b2ad-

In [34]:
results = vector_store.similarity_search(
    "I want to take a 400 level class relating to the environment which has a rate my professor score of 4 or higher",
    k=4,
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* COLLEGE OF THE ENVIRONMENT: C ENV [{}]
* ENVIRONMENTAL SCIENCE - UW BOTHELL: BES [{}]
* CIVIL & ENVIRONMENTAL ENGINEERING: CEE [{}]
* ENVIRONMENTAL SUSTAINABILITY: TEST [{}]
